In [22]:
import pandas as pd
import os
os.chdir('C:\\Users\\Shyam\\Documents\\GitHub\\beacondigest\\notebooks\\2021\\even more eip1559 paper\\random_forest_regression')

In [47]:
features = pd.read_csv('df_for_regression.csv')
features = features.iloc[: , 1:]

features.head(5)

,block_number,value,gas,is_to_address_uni_pool,is_sipher_inu_transaction,gas_used,is_to_uniswap_router
0,13188414,0.000000e+00,46703,False,False,46703,False
1,13188414,0.000000e+00,450000,False,False,246088,False
2,13188414,2.300000e+17,215887,False,False,163754,False
3,13188414,0.000000e+00,450000,False,False,246090,False
4,13188414,0.000000e+00,74926,False,False,74926,False


In [48]:
print('The shape of our features is:', features.shape)
features.iloc[: , -6:].describe()

The shape of our features is: (1258391, 7)


,value,gas,gas_used
count,1.258391e+06,1.258391e+06,1.258391e+06
mean,2.016572e+18,1.837131e+05,8.184644e+04
std,3.167764e+20,4.110996e+05,1.672691e+05
min,0.000000e+00,2.100000e+04,2.100000e+04
25%,0.000000e+00,4.200000e+04,2.100000e+04
50%,0.000000e+00,1.000000e+05,4.610900e+04
75%,1.000000e+17,2.325460e+05,8.651800e+04
max,2.235530e+23,3.000000e+07,2.308042e+07


In [49]:
# Use numpy to convert to arrays
import numpy as np

# Labels are the values we want to predict
labels = np.array(list(features['gas_used']))

# Remove the labels from the features
# axis 1 refers to the columns
features= features.drop('gas_used', axis = 1)
features= features.drop('block_number', axis = 1)
features= features.drop('value', axis = 1)
features= features.drop('is_to_address_uni_pool', axis = 1)

# Saving feature names for later use
feature_list = list(features.columns)

# Convert to numpy array
features = np.array(features)

In [26]:
# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 123)

In [27]:
print('Training Features Shape:', train_features.shape)
print('Training Labels Shape:', train_labels.shape)
print('Testing Features Shape:', test_features.shape)
print('Testing Labels Shape:', test_labels.shape)

Training Features Shape: (943793, 3)
Training Labels Shape: (943793,)
Testing Features Shape: (314598, 3)
Testing Labels Shape: (314598,)


In [28]:
baseline_predictions =  test_features[:, feature_list.index('gas')]
baseline_errors = abs(baseline_predictions - test_labels)
print('Average baseline error: ', round(np.mean(baseline_errors), 2))

Average baseline error:  101241.85


In [29]:
mape_baseline = 100 * (baseline_errors / test_labels)

# Calculate and display accuracy
baseline_accuracy = 100 - np.mean(mape_baseline)
print('Accuracy:', round(baseline_accuracy, 2), '%.')

Accuracy: -112.17 %.


In [30]:
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor

# Instantiate model with 100 decision trees
rf = RandomForestRegressor(n_estimators = 100, random_state = 123)

# Train the model on training data
rf.fit(train_features, train_labels);

In [36]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 18071.71


In [37]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 69.8 %.


In [ ]:
import pickle

filename = 'random_forest_model.sav'
pickle.dump(rf, open(filename, 'wb'))

In [15]:
loaded_model = pickle.load(open(filename, 'rb'))

In [ ]:
# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot

# Pull out one tree from the forest
tree = rf.estimators_[5]

# Import tools needed for visualization
from sklearn.tree import export_graphviz
import pydot

# Pull out one tree from the forest
tree = rf.estimators_[5]

# Export the image to a dot file
export_graphviz(tree, out_file = 'tree.dot', feature_names = feature_list, rounded = True, precision = 1)

# Use dot file to create a graph
(graph, ) = pydot.graph_from_dot_file('tree.dot')

# Write graph to a png file
graph.write_png('tree.png')

In [ ]:
# Limit depth of tree to 3 levels
rf_small = RandomForestRegressor(n_estimators=10, max_depth = 3)
rf_small.fit(train_features, train_labels)

# Extract the small tree
tree_small = rf_small.estimators_[5]

# Save the tree as a png image
export_graphviz(tree_small, out_file = 'small_tree.dot', feature_names = feature_list, rounded = True, precision = 1)
(graph, ) = pydot.graph_from_dot_file('small_tree.dot')
graph.write_png('small_tree.png');

In [32]:
# Get numerical feature importances
importances = list(rf.feature_importances_)

# List of tuples with variable and importance
feature_importances = [(feature, round(importance, 2)) for feature, importance in zip(feature_list, importances)]

# Sort the feature importances by most important first
feature_importances = sorted(feature_importances, key = lambda x: x[1], reverse = True)

# Print out the feature and importances 
[print('Variable: {:20} Importance: {}'.format(*pair)) for pair in feature_importances];

Variable: gas                  Importance: 0.99
Variable: is_to_uniswap_router Importance: 0.01
Variable: is_sipher_inu_transaction Importance: 0.0


In [33]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    "n_estimators":[100,200,300],
    "max_depth":[10, 50, 100],
}

rf_reg = RandomForestRegressor()

rf_reg_tuned = GridSearchCV(estimator=rf_reg,
                            param_grid=param_grid,
                            cv=3,
                            n_jobs=-1,
                            verbose=2)

rf_reg_tuned.fit(train_features,train_labels)
rf_reg_tuned.best_estimator_

Fitting 3 folds for each of 9 candidates, totalling 27 fits


RandomForestRegressor(max_depth=50)

In [34]:
# Use the forest's predict method on the test data
predictions = rf_reg_tuned.predict(test_features)

# Calculate the absolute errors
errors = abs(predictions - test_labels)

# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2))

Mean Absolute Error: 18084.88


In [35]:
# Calculate mean absolute percentage error (MAPE)
mape = 100 * (errors / test_labels)

# Calculate and display accuracy
accuracy = 100 - np.mean(mape)
print('Accuracy:', round(accuracy, 2), '%.')

Accuracy: 69.81 %.


## Predicting new datasets with trained model

In [96]:
features = pd.read_csv('sipher_inu-clean.csv', low_memory = False)
features.head(5)

,detecttime,hash,status,gas,gasprice,curblocknumber,timepending,blockspending,replace,fromaddress,toaddress,nonce,type,basefeepergas,maxpriorityfeepergas,maxfeepergas
0,2021-09-09T23:59:59.892Z,0x9689c224e6c5598595d306c450f5c7cd91db102abb5a...,pending,150000,1.522178e+11,13194682,NaN,NaN,NaN,0xcc4026ab5cfc652cabe9bce3a5f446dd3a62b31f,0x1b705bec5ca8406fd621ef09010a4c3a233e19eb,1,0,79910187954,NaN,NaN
1,2021-09-09T23:59:59.663Z,0xba059c12a51ce18b7cf529721ebd707a331960215580...,pending,123188,1.040000e+11,13194682,NaN,NaN,NaN,0x5785fd1aa3d09561e3ee3cb9c50ce94c66a6eec3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,14,0,79910187954,NaN,NaN
2,2021-09-09T23:59:59.640Z,0xa4ab274c3b8275e35824400cde3956f341cd7df6d95f...,pending,90000,1.071956e+11,13194682,NaN,NaN,NaN,0x4ad64983349c49defe8d7a4686202d24b25d0ce8,0x99d1de1982c5c9834d7efa5726460c8a092a88cc,233867,0,79910187954,NaN,NaN
3,2021-09-09T23:59:59.585Z,0x0a1e7f984b286a20c054c6cfb8ff9699476c7878c21e...,pending,154032,1.290029e+11,13194682,NaN,NaN,NaN,0x808b4da0be6c9512e948521452227efc619bea52,0x3fda25f27211a138adf211f4c060f2149674be6d,840603,0,99485418936,NaN,NaN
4,2021-09-09T23:59:59.585Z,0x97cb5e9532f54eff6737a9301a5ce78755c97206ae9d...,pending,179032,1.151150e+11,13194682,NaN,NaN,NaN,0x808b4da0be6c9512e948521452227efc619bea52,0x3fda25f27211a138adf211f4c060f2149674be6d,840604,0,99485418936,NaN,NaN


In [97]:
features['is_to_uniswap_router'] = [(to_address == '0x7a250d5630b4cf539739df2c5dacb4c659f2488d' or to_address == '0xe592427a0aece92de3edee1f18e0157c05861564') for to_address in list(features['toaddress'])]
features['is_nft_transaction'] = [to_address == '0x315bc085a14e251f129a361afa37205e3313bf15' for to_address in list(features['toaddress'])]
features.head(5)

,detecttime,hash,status,gas,gasprice,curblocknumber,timepending,blockspending,replace,fromaddress,toaddress,nonce,type,basefeepergas,maxpriorityfeepergas,maxfeepergas,is_to_uniswap_router,is_nft_transaction
0,2021-09-09T23:59:59.892Z,0x9689c224e6c5598595d306c450f5c7cd91db102abb5a...,pending,150000,1.522178e+11,13194682,NaN,NaN,NaN,0xcc4026ab5cfc652cabe9bce3a5f446dd3a62b31f,0x1b705bec5ca8406fd621ef09010a4c3a233e19eb,1,0,79910187954,NaN,NaN,False,False
1,2021-09-09T23:59:59.663Z,0xba059c12a51ce18b7cf529721ebd707a331960215580...,pending,123188,1.040000e+11,13194682,NaN,NaN,NaN,0x5785fd1aa3d09561e3ee3cb9c50ce94c66a6eec3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,14,0,79910187954,NaN,NaN,False,False
2,2021-09-09T23:59:59.640Z,0xa4ab274c3b8275e35824400cde3956f341cd7df6d95f...,pending,90000,1.071956e+11,13194682,NaN,NaN,NaN,0x4ad64983349c49defe8d7a4686202d24b25d0ce8,0x99d1de1982c5c9834d7efa5726460c8a092a88cc,233867,0,79910187954,NaN,NaN,False,False
3,2021-09-09T23:59:59.585Z,0x0a1e7f984b286a20c054c6cfb8ff9699476c7878c21e...,pending,154032,1.290029e+11,13194682,NaN,NaN,NaN,0x808b4da0be6c9512e948521452227efc619bea52,0x3fda25f27211a138adf211f4c060f2149674be6d,840603,0,99485418936,NaN,NaN,False,False
4,2021-09-09T23:59:59.585Z,0x97cb5e9532f54eff6737a9301a5ce78755c97206ae9d...,pending,179032,1.151150e+11,13194682,NaN,NaN,NaN,0x808b4da0be6c9512e948521452227efc619bea52,0x3fda25f27211a138adf211f4c060f2149674be6d,840604,0,99485418936,NaN,NaN,False,False


In [98]:
feature_set = features[['gas','is_nft_transaction','is_to_uniswap_router']]

feature_list = list(feature_set.columns)

# Convert to numpy array
feature_set = np.array(feature_set)

In [99]:
# Use the forest's predict method on the test data
predictions = rf.predict(feature_set)

In [100]:
features['gas_used_predicted'] = list(predictions)
features.to_csv('sipher_inu_predictions.csv')
features

,detecttime,hash,status,gas,gasprice,curblocknumber,timepending,blockspending,replace,fromaddress,toaddress,nonce,type,basefeepergas,maxpriorityfeepergas,maxfeepergas,is_to_uniswap_router,is_nft_transaction,gas_used_predicted
0,2021-09-09T23:59:59.892Z,0x9689c224e6c5598595d306c450f5c7cd91db102abb5a...,pending,150000,1.522178e+11,13194682,NaN,NaN,NaN,0xcc4026ab5cfc652cabe9bce3a5f446dd3a62b31f,0x1b705bec5ca8406fd621ef09010a4c3a233e19eb,1,0,79910187954,NaN,NaN,False,False,53270.382552
1,2021-09-09T23:59:59.663Z,0xba059c12a51ce18b7cf529721ebd707a331960215580...,pending,123188,1.040000e+11,13194682,NaN,NaN,NaN,0x5785fd1aa3d09561e3ee3cb9c50ce94c66a6eec3,0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48,14,0,79910187954,NaN,NaN,False,False,46451.913651
2,2021-09-09T23:59:59.640Z,0xa4ab274c3b8275e35824400cde3956f341cd7df6d95f...,pending,90000,1.071956e+11,13194682,NaN,NaN,NaN,0x4ad64983349c49defe8d7a4686202d24b25d0ce8,0x99d1de1982c5c9834d7efa5726460c8a092a88cc,233867,0,79910187954,NaN,NaN,False,False,31390.346567
3,2021-09-09T23:59:59.585Z,0x0a1e7f984b286a20c054c6cfb8ff9699476c7878c21e...,pending,154032,1.290029e+11,13194682,NaN,NaN,NaN,0x808b4da0be6c9512e948521452227efc619bea52,0x3fda25f27211a138adf211f4c060f2149674be6d,840603,0,99485418936,NaN,NaN,False,False,84035.023602
4,2021-09-09T23:59:59.585Z,0x97cb5e9532f54eff6737a9301a5ce78755c97206ae9d...,pending,179032,1.151150e+11,13194682,NaN,NaN,NaN,0x808b4da0be6c9512e948521452227efc619bea52,0x3fda25f27211a138adf211f4c060f2149674be6d,840604,0,99485418936,NaN,NaN,False,False,109035.892212
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1309232,2021-09-09T00:00:00.459Z,0x47f30381cdf796efd0ed88ae8db3ed4a4f86f244a488...,pending,316658,NaN,13188216,NaN,NaN,NaN,0xf711ae72a85573a15de36ae5b57c27d9f9e6fa77,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,30,2,105925666210,1.500000e+09,1.359806e+11,False,False,229013.360000
1309233,2021-09-09T00:00:00.401Z,0x2a9c3d5a50b07486db79df99678f8772743820e2d827...,pending,241000,1.290000e+11,13188216,NaN,NaN,NaN,0xec6770c4367aa1d722892f4d768f150f51246732,0x7be8076f4ea4a4ad08075c2508e481d6c946d12b,17,0,105925666210,NaN,NaN,False,False,186806.686112
1309234,2021-09-09T00:00:00.291Z,0xc6f6560612b3d29f4baa3ca60bf5c27f020bb5d56b60...,pending,2800000,1.446638e+11,13188216,NaN,NaN,NaN,0xd8aa8f3be2fb0c790d3579dcf68a04701c1e33db,0xd5090674b4653240cd94ee886484ca808c6e6694,298524,0,105925666210,NaN,NaN,False,False,45590.253502
1309235,2021-09-09T00:00:00.290Z,0x8135f8a108c5dfcdd869b3f9ccb7fa719fe5dc246e99...,pending,46258,NaN,13188216,NaN,NaN,NaN,0xdc87877616641771a2e9857ad4c098df6452f2b0,0xbe6e3669464e7db1e1528212f0bff5039461cb82,190,2,105925666210,1.500000e+09,1.477159e+11,False,False,44691.099671
